In [67]:
from __future__ import (
    absolute_import, absolute_import, division, division,
    print_function, print_function
)
import re

import nltk
from keras import metrics
nltk.download('stopwords')
from nltk.corpus import stopwords
import stanza
stanza.download('pt')

import matplotlib.pyplot as plt
from matplotlib import cm

import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import f_classif, SelectKBest
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.python.keras import models, metrics
from tensorflow.python.keras.layers import Dense, Dropout


FLAGS = None
NGRAM_RANGE = (1, 2)
TOP_K = 20000
TOKEN_MODE = 'word'
MIN_DOCUMENT_FREQUENCY = 2


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
2020-11-18 16:05:50 INFO: Downloading default packages for language: pt (Portuguese)...
2020-11-18 16:05:51 INFO: File exists: C:\Users\HC\stanza_resources\pt\default.zip.
2020-11-18 16:05:55 INFO: Finished downloading models and saved to C:\Users\HC\stanza_resources.


In [59]:
def read():
    df = pd.read_csv('dataset/Covid BR Tweets/opcovidbr.csv', index_col='Id')
    return train_test_split(df, test_size=0.3, random_state=123)

def remove_urls(text):
    url_remover = re.compile(r'https?://\S+|www\.\S+')
    return url_remover.sub(r'', text)

def remove_html(text):
    html_remover=re.compile(r'<.*?>')
    return html_remover.sub(r'',text)

def remove_mentions(text):
    mention_remover=re.compile(r'@\w+')
    return mention_remover.sub(r'',text)

def remove_numbers(text):
    number_remover=re.compile(r'\d+')
    return number_remover.sub(r'',text)

def remove_hashtags(text):
    number_remover=re.compile(r'#\w+')
    return number_remover.sub(r'',text)

def remove_punctuation(text):
    punct_remover=re.compile(r'[^\w\s\d]+')
    return punct_remover.sub(r'',text)

def remove_excessive_whitespace(text):
    ws_remover=re.compile(r'\s+')
    return ws_remover.sub(r' ', str(text)).strip()

def remove_stopwords(text, stop_words):


    return " ".join([word for word in text.split(" ") if word not in stop_words])

def lowering(text):
    return text.lower()

def lemmatization(text, nlp):
    doc = nlp(text)
    return ' '.join([f'{word.lemma}' for sent in doc.sentences for word in \
            sent.words])

def clean(df):
    stop_words = set(stopwords.words('portuguese'))
    df = df.loc[:,["twitter", "polarity"]]
    df = df[-df.polarity.isnull()]
    df["score"] = df.polarity
    df["score"] = df.score.apply(lambda x: 0 if x == -1 else 1)
    
    df["text"] = df.twitter
    df["text"] = df.text.apply(lambda x: remove_urls(x))
    df["text"] = df.text.apply(lambda x: remove_mentions(x))
    df["text"] = df.text.apply(lambda x: remove_html(x))
    df["text"] = df.text.apply(lambda x: remove_numbers(x))
    df["text"] = df.text.apply(lambda x: remove_hashtags(x))
    df["text"] = df.text.apply(lambda x: remove_punctuation(x))
    df["text"] = df.text.apply(lambda x: remove_excessive_whitespace(x))
    df["text"] = df.text.apply(lambda x: remove_stopwords(x, stop_words))
    df["text"] = df.text.apply(lambda x: lowering(x))
    # Removing messages that are too short.
    df = df[df.text.apply(lambda x: len(x.split(" ")) > 2)]

    nlp = stanza.Pipeline(lang='pt', processors='tokenize,mwt,pos,lemma')

    df["text"] = df.text.apply(lambda x: lemmatization(x, nlp))


    return df

def cleaning():
    df_train, df_test = read()
    df_train = clean(df_train)
    df_test = clean(df_test)

    return df_train, df_test


In [60]:
def ngram_vectorize(train_texts, train_labels, val_texts):
    # Create keyword arguments to pass to the 'tf-idf' vectorizer.
    kwargs = {
        'ngram_range': NGRAM_RANGE,
        'dtype': 'int32',
        'strip_accents': 'unicode',
        'decode_error': 'replace',
        'analyzer': TOKEN_MODE,
        'min_df': MIN_DOCUMENT_FREQUENCY,
    }
    vectorizer = TfidfVectorizer(**kwargs)

    # Learn vocabulary from training texts and vectorize training texts.
    x_train = vectorizer.fit_transform(train_texts)

    # Vectorize validation texts.
    x_val = vectorizer.transform(val_texts)

    # Select top 'k' of the vectorized features.
    selector = SelectKBest(f_classif, k=min(TOP_K, x_train.shape[1]))
    selector.fit(x_train, train_labels)
    x_train = selector.transform(x_train)
    x_val = selector.transform(x_val)

    x_train = x_train.astype('float32')
    x_val = x_val.astype('float32')
    return x_train, x_val

def mlp_model(layers, units, dropout_rate, input_shape):
    """Creates an instance of a multi-layer perceptron model.

    # Arguments
        layers: int, number of `Dense` layers in the model.
        units: int, output dimension of the layers.
        dropout_rate: float, percentage of input to drop at Dropout layers.
        input_shape: tuple, shape of input to the model.
        num_classes: int, number of output classes.

    # Returns
        An MLP model instance.
    """
    op_units, op_activation = 1, 'sigmoid'
    model = models.Sequential()
    model.add(Dropout(rate=dropout_rate, input_shape=input_shape))

    for _ in range(layers - 1):
        model.add(Dense(units=units, activation='relu'))
        model.add(Dropout(rate=dropout_rate))

    model.add(Dense(units=op_units, activation=op_activation))
    return model



def train_ngram_model(data,
                      learning_rate=1e-3,
                      epochs=1000,
                      batch_size=128,
                      layers=2,
                      units=64,
                      dropout_rate=0.2):
    # Get the data.
    (train_texts, train_labels), (val_texts, val_labels) = data

    # Vectorize texts.
    x_train, x_val = ngram_vectorize(train_texts, train_labels, val_texts)

    # Create model instance.
    model = mlp_model(layers=layers,
                      units=units,
                      dropout_rate=dropout_rate,
                      input_shape=x_train.shape[1:])

    # Compile model with learning parameters.
    loss = 'binary_crossentropy'
    optimizer = tf.keras.optimizers.Adam(lr=learning_rate)
    model.compile(optimizer=optimizer, loss=loss, 
                  metrics=['acc', 
                           metrics.AUC()])

    # Create callback for early stopping on validation loss. If the loss does
    # not decrease in two consecutive tries, stop training.
    callbacks = [tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', patience=2)]

    # Train and validate model.
    history = model.fit(
        x_train.toarray(),
        train_labels,
        epochs=epochs,
        callbacks=callbacks,
        validation_data=(x_val.toarray(), val_labels),
        verbose=1,  # Logs once per epoch.
        batch_size=batch_size)

    # Print results.
    history = history.history
    print('Validation accuracy: {acc}, loss: {loss}'.format(
        acc=history['val_acc'][-1], loss=history['val_loss'][-1]))

    # Save model.
    model.save('experimento_2_ii_mlp_model.h5')
    return history['val_acc'][-1], history['val_loss'][-1]

In [61]:
df_train, df_test = cleaning()

2020-11-18 16:45:28 INFO: Loading these models for language: pt (Portuguese):
| Processor | Package |
-----------------------
| tokenize  | bosque  |
| mwt       | bosque  |
| pos       | bosque  |
| lemma     | bosque  |

2020-11-18 16:45:28 INFO: Use device: cpu
2020-11-18 16:45:28 INFO: Loading: tokenize
2020-11-18 16:45:37 INFO: Loading: mwt
2020-11-18 16:45:37 INFO: Loading: pos
2020-11-18 16:45:41 INFO: Loading: lemma
2020-11-18 16:45:41 INFO: Done loading processors!
2020-11-18 16:47:06 INFO: Loading these models for language: pt (Portuguese):
| Processor | Package |
-----------------------
| tokenize  | bosque  |
| mwt       | bosque  |
| pos       | bosque  |
| lemma     | bosque  |

2020-11-18 16:47:06 INFO: Use device: cpu
2020-11-18 16:47:06 INFO: Loading: tokenize
2020-11-18 16:47:06 INFO: Loading: mwt
2020-11-18 16:47:06 INFO: Loading: pos
2020-11-18 16:47:08 INFO: Loading: lemma
2020-11-18 16:47:08 INFO: Done loading processors!


In [62]:
print("Reading")
data = (df_train.text, df_train.score), (df_test.text, df_test.score)
print("Training")
train_ngram_model(data, epochs=1000)

Reading
Training
Train on 417 samples, validate on 180 samples
Epoch 1/1000
417/417 [==============================] - 4s 8ms/sample - loss: 0.6951 - acc: 0.4868 - auc_16: 0.4455 - val_loss: 0.6935 - val_acc: 0.4667 - val_auc_16: 0.4864
Epoch 2/1000
417/417 [==============================] - 0s 278us/sample - loss: 0.6862 - acc: 0.6595 - auc_16: 0.7024 - val_loss: 0.6917 - val_acc: 0.5222 - val_auc_16: 0.5707
Epoch 3/1000
417/417 [==============================] - 0s 290us/sample - loss: 0.6768 - acc: 0.7770 - auc_16: 0.8584 - val_loss: 0.6896 - val_acc: 0.5611 - val_auc_16: 0.6506
Epoch 4/1000
417/417 [==============================] - 0s 624us/sample - loss: 0.6694 - acc: 0.8106 - auc_16: 0.9246 - val_loss: 0.6875 - val_acc: 0.5889 - val_auc_16: 0.6906
Epoch 5/1000
417/417 [==============================] - 0s 240us/sample - loss: 0.6594 - acc: 0.8777 - auc_16: 0.9629 - val_loss: 0.6855 - val_acc: 0.6000 - val_auc_16: 0.7050
Epoch 6/1000
417/417 [==============================] - 0s 

C:\Users\HC\miniconda3\envs\SentiHIV\lib\site-packages\sklearn\feature_extraction\text.py:1799: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. int32 'dtype' will be converted to np.float64.
  UserWarning)


(0.6722222, 0.5778663039207459)

In [63]:
def tune_ngram_model(data):
    """Tunes n-gram model on the given dataset.
    # Arguments
        data: tuples of training and test texts and labels.
    """
    # Select parameter values to try.
    num_layers = [1, 2, 3]
    num_units = [8, 16, 32, 64, 128]

    # Save parameter combination and results.
    params = {
        'layers': [],
        'units': [],
        'accuracy': [],
    }

    # Iterate over all parameter combinations.
    for layers in num_layers:
        for units in num_units:
                params['layers'].append(layers)
                params['units'].append(units)

                accuracy, _ = train_ngram_model(
                    data=data,
                    layers=layers,
                    units=units)
                print(('Accuracy: {accuracy}, Parameters: (layers={layers}, '
                       'units={units})').format(accuracy=accuracy,
                                                layers=layers,
                                                units=units))
                params['accuracy'].append(accuracy)
    # _plot_parameters(params)
    return params

def _plot_parameters(params):
    """Creates a 3D surface plot of given parameters.
    # Arguments
        params: dict, contains layers, units and accuracy value combinations.
    """
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    ax.plot_trisurf(params['layers'],
                    params['units'],
                    params['accuracy'],
                    cmap=cm.coolwarm,
                    antialiased=False)
    plt.show()

In [68]:
tuned_params = tune_ngram_model(data)

C:\Users\HC\miniconda3\envs\SentiHIV\lib\site-packages\sklearn\feature_extraction\text.py:1799: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. int32 'dtype' will be converted to np.float64.
  UserWarning)
C:\Users\HC\miniconda3\envs\SentiHIV\lib\site-packages\sklearn\feature_extraction\text.py:1799: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. int32 'dtype' will be converted to np.float64.
  UserWarning)
C:\Users\HC\miniconda3\envs\SentiHIV\lib\site-packages\sklearn\feature_extraction\text.py:1799: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. int32 'dtype' will be converted to np.float64.
  UserWarning)
C:\Users\HC\miniconda3\envs\SentiHIV\lib\site-packages\sklearn\feature_extraction\text.py:1799: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 

Train on 417 samples, validate on 180 samples
Epoch 1/1000
417/417 [==============================] - 3s 8ms/sample - loss: 0.6948 - acc: 0.4916 - auc_32: 0.4747 - val_loss: 0.6912 - val_acc: 0.5278 - val_auc_32: 0.5731
Epoch 2/1000
417/417 [==============================] - 0s 537us/sample - loss: 0.6919 - acc: 0.5348 - auc_32: 0.5288 - val_loss: 0.6907 - val_acc: 0.5500 - val_auc_32: 0.5766
Epoch 3/1000
417/417 [==============================] - 0s 432us/sample - loss: 0.6904 - acc: 0.5348 - auc_32: 0.5615 - val_loss: 0.6902 - val_acc: 0.5778 - val_auc_32: 0.6017
Epoch 4/1000
417/417 [==============================] - 0s 765us/sample - loss: 0.6876 - acc: 0.6019 - auc_32: 0.6364 - val_loss: 0.6897 - val_acc: 0.5833 - val_auc_32: 0.6114
Epoch 5/1000
417/417 [==============================] - 0s 499us/sample - loss: 0.6865 - acc: 0.6139 - auc_32: 0.6662 - val_loss: 0.6892 - val_acc: 0.6167 - val_auc_32: 0.6169
Epoch 6/1000
417/417 [==============================] - 0s 223us/sample - lo

In [38]:
ngram_train, ngram_test = ngram_vectorize(df_train.text, df_train.score, df_test.text)

C:\Users\HC\miniconda3\envs\SentiHIV\lib\site-packages\sklearn\feature_extraction\text.py:1799: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. int32 'dtype' will be converted to np.float64.
  UserWarning)


3.385676220059395

In [70]:
pd.DataFrame(tuned_params).sort_values('accuracy', ascending=False)

,layers,units,accuracy
3,1,64,0.705556
0,1,8,0.700000
2,1,32,0.700000
4,1,128,0.700000
6,2,16,0.700000
9,2,128,0.694444
1,1,16,0.688889
5,2,8,0.688889
8,2,64,0.688889
13,3,64,0.688889
